In [5]:
import sys

# 动态添加test_package文件夹的路径，为了能让此文件夹下的
# 自定义包成功的导入
# 要根据你自己的实际包的模块来决定路径。


if sys.path[-1] != '../../../' :
    sys.path.append('../')
    sys.path.append('../../')
    sys.path.append('../../../')
else:
    pass

# 打印所有python解释器可以搜索到的所有路径
print(sys.path)

['', 'D:\\Python\\Anaconda3\\python36.zip', 'D:\\Python\\Anaconda3\\DLLs', 'D:\\Python\\Anaconda3\\lib', 'D:\\Python\\Anaconda3', 'C:\\Users\\Yukirito\\AppData\\Roaming\\Python\\Python36\\site-packages', 'D:\\Python\\Anaconda3\\lib\\site-packages', 'D:\\Python\\Anaconda3\\lib\\site-packages\\win32', 'D:\\Python\\Anaconda3\\lib\\site-packages\\win32\\lib', 'D:\\Python\\Anaconda3\\lib\\site-packages\\Pythonwin', 'D:\\Python\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Yukirito\\.ipython', '../', '../../', '../../../']


## 5.9-scikit-learn中的回归问题

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
'''
这一次我们不止取一个boston.data的一个特征, 而是都取了出来
'''
boston = datasets.load_boston()

X = boston.data
y = boston.target

X = X[y < 50.0]
y = y[y < 50.0]

In [3]:
X.shape

(490, 13)

In [6]:
from playML.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, seed=666)

### scikit-learn 中的线性回归

In [7]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

In [8]:
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [9]:
'''
查看系数
'''
lin_reg.coef_

array([-1.18919477e-01,  3.63991462e-02, -3.56494193e-02,  5.66737830e-02,
       -1.16195486e+01,  3.42022185e+00, -2.31470282e-02, -1.19509560e+00,
        2.59339091e-01, -1.40112724e-02, -8.36521175e-01,  7.92283639e-03,
       -3.81966137e-01])

In [10]:
'''
查看截距
'''
lin_reg.intercept_

34.16143549624665

In [11]:
lin_reg.score(X_test, y_test)

0.8129802602658495

### kNN Regressor

In [12]:
from sklearn.neighbors import KNeighborsRegressor

knn_reg = KNeighborsRegressor()
knn_reg.fit(X_train, y_train)
knn_reg.score(X_test, y_test)

0.5865412198300899

In [13]:
'''
初始k=5
score并不理想

我们来进行一下超参数调优
'''
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        'weights': ['uniform'],
        'n_neighbors': [i for i in range(1, 11)]
    },
    {
        'weights': ['distance'],
        'n_neighbors': [i for i in range(1, 11)],
        'p': [i for i in range(1, 6)]
    },
]

knn_reg = KNeighborsRegressor()
grid_search = GridSearchCV(knn_reg, param_grid, n_jobs=-1, verbose=50)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 60 candidates, totalling 180 fits
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   11.7s
[Paralle

GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'weights': ['uniform'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}, {'weights': ['distance'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=50)

In [14]:
grid_search.best_params_

{'n_neighbors': 5, 'p': 1, 'weights': 'distance'}

In [15]:
'''
在这里我们会发现 0.63 和 0.58差不多
但是其实他们的score判断方法是不一样得

网格搜索使用了 CV交叉验证 的方式
'''
grid_search.best_score_

0.634093080186858

In [17]:
'''
如果我们要得到和前面数据同样的衡量标准的score

基于网格搜索 
'''
grid_search.best_estimator_.score(X_test, y_test)

0.7044357727037996